In [1]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition, manifold

% matplotlib notebook

In [2]:
def compute_distance(x,y):
    x = x / np.linalg.norm(x)
    y = y / np.linalg.norm(y)
    return np.linalg.norm(x-y)

In [3]:
def print_percentage(n, t):
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('=' * ((n * 20/t) + 1) , n * 100/t + 1 ))
    sys.stdout.flush()

In [4]:
def norm_feat(feat):
    norms = np.linalg.norm( feat, axis=1)
    feat = (feat.T / norms.T).T
    return feat[ ~np.isnan(feat[:,0]) ]

In [ ]:
feature_dir = '/fileserver/nmec-handwriting/localfeatures/nmec_bw_denoised_cc_deNNiam_fiel657_min500/'
#feature_dir = '/fileserver/nmec-handwriting/localfeatures/nmec_bw_cc_deNNiam_fiel657_min500'
#feature_dir = '/fileserver/nmec-handwriting/localfeatures/nmec_bw_crop_cc_deNNiam120_fiel657-120'

files = os.listdir(feature_dir)
files.sort()
files = files[:-1]

C = np.zeros((len(files),len(files)))
feature_map = []
for i,filei in enumerate(files):
    for j, filej in enumerate(files):
        # Only calculate upper right hand triangle
        if j <= i:
            C[i,j] = -1
            continue
        feati = norm_feat( np.load(feature_dir + "/" + filei) )
        featj = norm_feat( np.load(feature_dir + "/" + filej) )
        Cij = np.exp( -feati.dot(featj.T) )
        C[i,j] = Cij.sum() / np.prod(Cij.shape)
    print_percentage(i, len(files))

[=                   ] 1%

In [ ]:
a = transform[0]
b = transform[1]
compute_min_distance(a,b)

In [ ]:
transform = feature_map

In [ ]:
metric = []
for i, image in enumerate(transform):
    metricline = [np.array([compute_distance(image, other) for other in transform])]
    metric += metricline
    print_percentage(i, len(transform))

metric = np.array(metric)
F = -metric
np.fill_diagonal(F, -sys.maxint)

In [ ]:
soft_correct = 0
hard_correct = 0
total_num = 0

k = 10
g = 8
max_top = 3

for j, i in enumerate(F):
    
    total_num += 1
    topk = i.argsort()[-k:]
    
    if files[j][:6] in (files[index][:6] for index in topk):
        soft_correct += 1
    
    hardsample = list(files[index][3:6] for index in topk[-max_top:])
    if len(set(hardsample)) == 1 and hardsample[0] == files[j][3:6]:
        print "%s matched %s" % (files[j][3:10], hardsample)
        hard_correct += 1

print "%-30s" % ( "-" * 37 )
print "SOFT CRITERIA: Top %d\t= %f" %(k, (soft_correct + 0.0) / total_num)
print "HARD CRITERIA: Top %d\t= %f" %(max_top, (hard_correct + 0.0) / total_num)